In [158]:
!pip install nltk gensim


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [159]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oskarroeske/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# EDA

In [160]:
df = pd.read_csv("../data/gpt_fake_recommendations.csv")

In [161]:
df.head()

,Company,Short Name,Date,Target Price,Recommendation,Commentary
0,Apple Inc.,AAPL,2023-01-15,121.35,Hold,"Despite promising growth in cost management, c..."
1,Apple Inc.,AAPL,2023-02-15,371.59,Buy,AAPL is benefiting from investment in research...
2,Apple Inc.,AAPL,2023-03-15,202.52,Hold,AAPL's performance in technology has been stea...
3,Apple Inc.,AAPL,2023-04-15,355.88,Hold,AAPL faces uncertainties in shift in consumer ...
4,Apple Inc.,AAPL,2023-05-15,292.45,Sell,AAPL's challenges in expansion into internatio...


In [162]:
df.value_counts("Recommendation")

Recommendation
Buy     434
Sell    428
Hold    374
Name: count, dtype: int64

In [163]:
df.value_counts("Company")

Company
3M Company                           12
Norfolk Southern Corporation         12
Salesforce, Inc.                     12
S&P Global Inc.                      12
Raytheon Technologies Corporation    12
                                     ..
Exelon Corporation                   12
Eli Lilly and Company                12
Dow Inc.                             12
Deere & Company                      12
Zoetis Inc.                          12
Name: count, Length: 103, dtype: int64

In [164]:
df.groupby(["Short Name"]).mean("Target Price")

,Target Price
Short Name,
AAPL,249.782500
ABBV,227.696667
ADBE,251.938333
ALL,280.011667
AMGN,294.452500
...,...
VZ,259.306667
WFC,246.332500
WMT,251.254167


In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Company         1236 non-null   object 
 1   Short Name      1236 non-null   object 
 2   Date            1236 non-null   object 
 3   Target Price    1236 non-null   float64
 4   Recommendation  1236 non-null   object 
 5   Commentary      1236 non-null   object 
dtypes: float64(1), object(5)
memory usage: 58.1+ KB


# Removing Stopwords

In [166]:
df["Commentary"] = df["Commentary"].apply(strip_multiple_whitespaces)

In [167]:
# Show list of stopwords
nltk_stopwords = set(stopwords.words('english'))
print(sorted(nltk_stopwords))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some',

In [168]:
# Remove negations from the list
to_remove=['ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'don', "don't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [169]:
for word in to_remove:
    nltk_stopwords.remove(word)

In [170]:
print(sorted(nltk_stopwords))

['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'd', 'did', 'do', 'does', 'doing', 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'more', 'most', 'my', 'myself', 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'she', "she's", 'should', "should've", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'we', 'were', 'what', 'when', 'where', 'which', 'while', 'who', 'who

In [171]:
# Correct apply version
df["Commentary"] = df["Commentary"].apply(lambda comment: remove_stopwords(comment, stopwords=nltk_stopwords))

In [172]:
print(df)

                 Company Short Name        Date  Target Price Recommendation  \
0             Apple Inc.       AAPL  2023-01-15        121.35           Hold   
1             Apple Inc.       AAPL  2023-02-15        371.59            Buy   
2             Apple Inc.       AAPL  2023-03-15        202.52           Hold   
3             Apple Inc.       AAPL  2023-04-15        355.88           Hold   
4             Apple Inc.       AAPL  2023-05-15        292.45           Sell   
...                  ...        ...         ...           ...            ...   
1231  Exelon Corporation        EXC  2023-08-15        242.37           Sell   
1232  Exelon Corporation        EXC  2023-09-15        248.48            Buy   
1233  Exelon Corporation        EXC  2023-10-15        148.36           Hold   
1234  Exelon Corporation        EXC  2023-11-15        143.41            Buy   
1235  Exelon Corporation        EXC  2023-12-15        172.69            Buy   

                                       

# Add dates for 3 / 6 / 9 / 12 months

In [173]:
# transform data column to actual date
df["Date"] = pd.to_datetime(df["Date"],format='%Y-%m-%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Company         1236 non-null   object        
 1   Short Name      1236 non-null   object        
 2   Date            1236 non-null   datetime64[ns]
 3   Target Price    1236 non-null   float64       
 4   Recommendation  1236 non-null   object        
 5   Commentary      1236 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 58.1+ KB


In [174]:
# Apply DateOffset for each row using 'apply' on the DataFrame
df["Date next Day"] = df["Date"].apply(lambda date: date + pd.DateOffset(days=1))
df["Date after 3 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=3))
df["Date after 6 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=6))
df["Date after 9 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=9))
df["Date after 12 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=12))


In [175]:
df

,Company,Short Name,Date,Target Price,Recommendation,Commentary,Date next Day,Date after 3 Months,Date after 6 Months,Date after 9 Months,Date after 12 Months
0,Apple Inc.,AAPL,2023-01-15,121.35,Hold,"Despite promising growth cost management, conc...",2023-01-16,2023-04-15,2023-07-15,2023-10-15,2024-01-15
1,Apple Inc.,AAPL,2023-02-15,371.59,Buy,AAPL benefiting investment research developmen...,2023-02-16,2023-05-15,2023-08-15,2023-11-15,2024-02-15
2,Apple Inc.,AAPL,2023-03-15,202.52,Hold,"AAPL's performance technology steady, risks su...",2023-03-16,2023-06-15,2023-09-15,2023-12-15,2024-03-15
3,Apple Inc.,AAPL,2023-04-15,355.88,Hold,AAPL faces uncertainties shift consumer prefer...,2023-04-16,2023-07-15,2023-10-15,2024-01-15,2024-04-15
4,Apple Inc.,AAPL,2023-05-15,292.45,Sell,AAPL's challenges expansion international mark...,2023-05-16,2023-08-15,2023-11-15,2024-02-15,2024-05-15
...,...,...,...,...,...,...,...,...,...,...,...
1231,Exelon Corporation,EXC,2023-08-15,242.37,Sell,With ongoing issues like regulatory pressure e...,2023-08-16,2023-11-15,2024-02-15,2024-05-15,2024-08-15
1232,Exelon Corporation,EXC,2023-09-15,248.48,Buy,EXC's continued growth environmental regulatio...,2023-09-16,2023-12-15,2024-03-15,2024-06-15,2024-09-15
1233,Exelon Corporation,EXC,2023-10-15,148.36,Hold,EXC faces uncertainties shift consumer prefere...,2023-10-16,2024-01-15,2024-04-15,2024-07-15,2024-10-15
1234,Exelon Corporation,EXC,2023-11-15,143.41,Buy,EXC benefiting labor shortages strong demand. ...,2023-11-16,2024-02-15,2024-05-15,2024-08-15,2024-11-15


# API Connection

In [199]:
"""import yfinance as yf
from datetime import timedelta
import pandas as pd
import numpy as np  # Import numpy to use np.abs


#this function might not be necessary since it makes more sense to look at the whole time frame to see if a target price was reached or not!

def get_stock_price_nearest(ticker, date):
    try:
        # Ensure the date is in Pandas Timestamp format
        target_date = pd.to_datetime(date)

        # Define a range around the target date to fetch data
        start_date = target_date - timedelta(days=5)
        end_date = target_date + timedelta(days=5)

        # Download data within this range
        stock_data = yf.download(ticker, start=start_date, end=end_date)

        if not stock_data.empty:
            # Check if data exists for the exact date
            if target_date in stock_data.index:
                return stock_data.loc[target_date]['Close']
            else:
                # Manually find the closest available trading day using numpy.abs()
                stock_data['diff'] = np.abs(stock_data.index - target_date)
                closest_date = stock_data['diff'].idxmin()  # Get the index of the closest date
                closest_price = stock_data.loc[closest_date]['Close']
                return closest_price
        else:
            return None
    
    except Exception as e:
        return f"An unexpected error occurred for {ticker}: {str(e)}"

#def get_max_stock_price_for_timeframe(short_name,time_frame):
    """

In [262]:
import pandas as pd

# Load and ensure 'Date' is in datetime format
df_performance_data = pd.read_csv("../data/performance_data.csv")

pd.to_datetime(df_performance_data['Date']).apply(lambda x: x.date())


# Set 'Date' as the index to enable date filtering
df_performance_data = df_performance_data.set_index('Date')


# Function to get the maximum stock price within a date range
def get_max_stock_price(ticker, start_date, end_date=None):
    try:
        # Ensure the dates are in Pandas Timestamp format
        #start_date = pd.to_datetime(start_date)
        if end_date is not None:
            end_date = end_date
        else:
            end_date = start_date  # If no end_date, use start_date for a single day

        filtered_data = df_performance_data.loc[(df_performance_data.index >= start_date) & 
                                                (df_performance_data.index <= end_date), ticker]
        
        if not filtered_data.empty:
            # Return the maximum price within the filtered date range
            max_price = filtered_data.max()
            return max_price
        else:
            return f"No data available for {ticker} between {start_date} and {end_date}"
    
    except KeyError:
        return f"Ticker {ticker} not found in the dataset."
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

In [260]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Company                    1236 non-null   object        
 1   Short Name                 1236 non-null   object        
 2   Date                       1236 non-null   datetime64[ns]
 3   Target Price               1236 non-null   float64       
 4   Recommendation             1236 non-null   object        
 5   Commentary                 1236 non-null   object        
 6   Date next Day              1236 non-null   datetime64[ns]
 7   Date after 3 Months        1236 non-null   datetime64[ns]
 8   Date after 6 Months        1236 non-null   datetime64[ns]
 9   Date after 9 Months        1236 non-null   datetime64[ns]
 10  Date after 12 Months       1236 non-null   datetime64[ns]
 11  Start Price                1236 non-null   object        
 12  One da

In [256]:
print(get_max_stock_price("AAPL","2023-01-15","2023-04-15"))

An unexpected error occurred: '>=' not supported between instances of 'str' and 'Timestamp'


In [257]:
df_performance_data

,AAPL,ABBV,ADBE,ADI,ADP,ADSK,AIG,AIZ,AKAM,ALGN,...,WBA,WFC,WM,WMB,WMT,WRB,XEL,XOM,XPO,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2022-10-24 00:00:00+00:00,147.812256,139.916351,316.220001,139.433075,226.744476,207.089996,51.947655,144.978622,86.389999,211.389999,...,30.465139,42.670929,157.501251,28.056877,45.159691,46.274643,57.420830,99.696877,28.086750,146.603928
2022-10-25 00:00:00+00:00,150.670578,138.924179,323.790009,141.208191,227.557983,215.720001,51.890305,147.029007,88.449997,217.979996,...,31.194052,42.831413,159.785995,28.733494,45.373489,44.042854,58.185448,99.023499,28.740345,149.479736
2022-10-26 00:00:00+00:00,147.713364,141.418549,320.480011,136.394150,225.050430,214.559998,51.823402,131.636856,86.760002,221.639999,...,31.185270,43.142952,154.868027,29.004141,45.720093,44.635876,58.343956,100.201904,29.916815,148.861359
2022-10-27 00:00:00+00:00,143.213226,142.336563,318.649994,135.718842,224.887726,210.149994,52.205723,127.247368,87.580002,181.529999,...,31.325783,43.095753,153.251297,29.319889,45.587284,45.369183,58.978035,100.585358,29.364231,148.341187
2022-10-28 00:00:00+00:00,154.033340,136.874893,325.679993,139.770752,232.228622,216.389999,54.289360,129.653870,89.209999,188.619995,...,32.124962,43.756580,152.873734,29.473255,46.163883,46.721008,60.954853,103.531364,30.106953,150.441574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-15 00:00:00+00:00,233.850006,191.860001,508.029999,225.869995,290.040009,286.480011,76.980003,194.789993,103.290001,219.149994,...,10.420000,63.029999,213.789993,50.900002,81.650002,58.919998,63.430000,120.349998,115.370003,192.389999
2024-10-16 00:00:00+00:00,231.779999,190.460007,502.540009,227.490005,290.700012,289.190002,77.139999,196.860001,103.580002,216.050003,...,11.070000,63.889999,213.139999,51.369999,81.220001,59.279999,64.010002,120.660004,116.830002,194.440002
2024-10-17 00:00:00+00:00,232.149994,188.570007,496.829987,227.339996,291.959991,290.570007,79.190002,198.160004,104.730003,216.020004,...,10.660000,64.379997,213.080002,51.540001,80.889999,61.040001,63.250000,120.349998,114.269997,191.000000


In [263]:
def calculate_prices(row):
    short_name = row['Short Name']
    row['Start Price'] = get_max_stock_price(short_name, row['Date'])
    row['One day after'] = get_max_stock_price(short_name,row['Date next Day'])
    row['Price after 3 Months'] = get_max_stock_price(short_name, row['Date'],row['Date after 3 Months'])
    row['Price after 6 Months'] = get_max_stock_price(short_name,row['Date after 3 Months'], row['Date after 6 Months'])
    row['Price after 9 Months'] = get_max_stock_price(short_name, row['Date after 6 Months'], row['Date after 9 Months'])
    #row['Price after 12 Months'] = get_max_stock_price(short_name, row['Date after 12 Months'])
    return row
# Apply the function to each row
df = df.apply(calculate_prices, axis=1)

In [252]:
df

,Company,Short Name,Date,Target Price,Recommendation,Commentary,Date next Day,Date after 3 Months,Date after 6 Months,Date after 9 Months,Date after 12 Months,Start Price,One day after,Price after 3 Months,Price after 6 Months,Price after 9 Months,tp_reached_after_3_months,tp_reached_after_6_months,tp_reached_after_9_months
0,Apple Inc.,AAPL,2023-01-15,121.35,Hold,"Despite promising growth cost management, conc...",2023-01-16,2023-04-15,2023-07-15,2023-10-15,2024-01-15,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,True,True,True
1,Apple Inc.,AAPL,2023-02-15,371.59,Buy,AAPL benefiting investment research developmen...,2023-02-16,2023-05-15,2023-08-15,2023-11-15,2024-02-15,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,False,False,False
2,Apple Inc.,AAPL,2023-03-15,202.52,Hold,"AAPL's performance technology steady, risks su...",2023-03-16,2023-06-15,2023-09-15,2023-12-15,2024-03-15,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,False,False,False
3,Apple Inc.,AAPL,2023-04-15,355.88,Hold,AAPL faces uncertainties shift consumer prefer...,2023-04-16,2023-07-15,2023-10-15,2024-01-15,2024-04-15,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,False,False,False
4,Apple Inc.,AAPL,2023-05-15,292.45,Sell,AAPL's challenges expansion international mark...,2023-05-16,2023-08-15,2023-11-15,2024-02-15,2024-05-15,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,Exelon Corporation,EXC,2023-08-15,242.37,Sell,With ongoing issues like regulatory pressure e...,2023-08-16,2023-11-15,2024-02-15,2024-05-15,2024-08-15,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,False,False,False
1232,Exelon Corporation,EXC,2023-09-15,248.48,Buy,EXC's continued growth environmental regulatio...,2023-09-16,2023-12-15,2024-03-15,2024-06-15,2024-09-15,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,False,False,False
1233,Exelon Corporation,EXC,2023-10-15,148.36,Hold,EXC faces uncertainties shift consumer prefere...,2023-10-16,2024-01-15,2024-04-15,2024-07-15,2024-10-15,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support...,False,False,False
1234,Exelon Corporation,EXC,2023-11-15,143.41,Buy,EXC benefiting labor shortages strong demand. ...,2023-11-16,2024-02-15,2024-05-15,2024-08-15,2024-11-15,An unexpected error occurred: '>=' not support...,An unexpected error occurred: '>=' not support..

In [253]:
print(df_performance_data.dtypes)


AAPL    float64
ABBV    float64
ADBE    float64
ADI     float64
ADP     float64
         ...   
WRB     float64
XEL     float64
XOM     float64
XPO     float64
ZTS     float64
Length: 207, dtype: object


In [225]:
df["Start Price"]

0       An unexpected error occurred: Invalid comparis...
1       An unexpected error occurred: Invalid comparis...
2       An unexpected error occurred: Invalid comparis...
3       An unexpected error occurred: Invalid comparis...
4       An unexpected error occurred: Invalid comparis...
                              ...                        
1231    An unexpected error occurred: Invalid comparis...
1232    An unexpected error occurred: Invalid comparis...
1233    An unexpected error occurred: Invalid comparis...
1234    An unexpected error occurred: Invalid comparis...
1235    An unexpected error occurred: Invalid comparis...
Name: Start Price, Length: 1236, dtype: object

In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Company               1236 non-null   object        
 1   Short Name            1236 non-null   object        
 2   Date                  1236 non-null   datetime64[ns]
 3   Target Price          1236 non-null   float64       
 4   Recommendation        1236 non-null   object        
 5   Commentary            1236 non-null   object        
 6   Date next Day         1236 non-null   datetime64[ns]
 7   Date after 3 Months   1236 non-null   datetime64[ns]
 8   Date after 6 Months   1236 non-null   datetime64[ns]
 9   Date after 9 Months   1236 non-null   datetime64[ns]
 10  Date after 12 Months  1236 non-null   datetime64[ns]
 11  Start Price           1224 non-null   float64       
 12  One day after         1224 non-null   float64       
 13  Price after 3 Mont

In [182]:
testing_df = df

In [183]:
testing_df.columns

Index(['Company', 'Short Name', 'Date', 'Target Price', 'Recommendation',
       'Commentary', 'Date next Day', 'Date after 3 Months',
       'Date after 6 Months', 'Date after 9 Months', 'Date after 12 Months',
       'Start Price', 'One day after', 'Price after 3 Months',
       'Price after 6 Months', 'Price after 9 Months'],
      dtype='object')

In [146]:
"""
# List of columns to check
columns_to_check = ["Price after 3 Months", "Price after 6 Months", "Price after 9 Months"]

# Drop rows where any of the columns has a 0
df = df.dropna(df[columns_to_check])
"""

TypeError: DataFrame.dropna() takes 1 positional argument but 2 were given

In [185]:
testing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Company               1236 non-null   object        
 1   Short Name            1236 non-null   object        
 2   Date                  1236 non-null   datetime64[ns]
 3   Target Price          1236 non-null   float64       
 4   Recommendation        1236 non-null   object        
 5   Commentary            1236 non-null   object        
 6   Date next Day         1236 non-null   datetime64[ns]
 7   Date after 3 Months   1236 non-null   datetime64[ns]
 8   Date after 6 Months   1236 non-null   datetime64[ns]
 9   Date after 9 Months   1236 non-null   datetime64[ns]
 10  Date after 12 Months  1236 non-null   datetime64[ns]
 11  Start Price           1224 non-null   float64       
 12  One day after         1224 non-null   float64       
 13  Price after 3 Mont

# Calculate performance of Target Prices

In [186]:
testing_df["tp_reached_after_3_months"] = testing_df["Target Price"] <= testing_df["Price after 3 Months"]
testing_df["tp_reached_after_6_months"] = testing_df["Target Price"] <= testing_df["Price after 6 Months"]
testing_df["tp_reached_after_9_months"] = testing_df["Target Price"] <= testing_df["Price after 9 Months"]

In [187]:
testing_df

,Company,Short Name,Date,Target Price,Recommendation,Commentary,Date next Day,Date after 3 Months,Date after 6 Months,Date after 9 Months,Date after 12 Months,Start Price,One day after,Price after 3 Months,Price after 6 Months,Price after 9 Months,tp_reached_after_3_months,tp_reached_after_6_months,tp_reached_after_9_months
0,Apple Inc.,AAPL,2023-01-15,121.35,Hold,"Despite promising growth cost management, conc...",2023-01-16,2023-04-15,2023-07-15,2023-10-15,2024-01-15,134.759995,135.940002,165.210007,190.690002,178.720001,True,True,True
1,Apple Inc.,AAPL,2023-02-15,371.59,Buy,AAPL benefiting investment research developmen...,2023-02-16,2023-05-15,2023-08-15,2023-11-15,2024-02-15,155.330002,153.710007,172.070007,177.449997,188.009995,False,False,False
2,Apple Inc.,AAPL,2023-03-15,202.52,Hold,"AAPL's performance technology steady, risks su...",2023-03-16,2023-06-15,2023-09-15,2023-12-15,2024-03-15,152.990005,155.850006,186.009995,175.009995,197.570007,False,False,False
3,Apple Inc.,AAPL,2023-04-15,355.88,Hold,AAPL faces uncertainties shift consumer prefer...,2023-04-16,2023-07-15,2023-10-15,2024-01-15,2024-04-15,165.210007,165.229996,190.690002,178.720001,183.630005,False,False,False
4,Apple Inc.,AAPL,2023-05-15,292.45,Sell,AAPL's challenges expansion international mark...,2023-05-16,2023-08-15,2023-11-15,2024-02-15,2024-05-15,172.070007,172.070007,177.449997,188.009995,183.860001,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,Exelon Corporation,EXC,2023-08-15,242.37,Sell,With ongoing issues like regulatory pressure e...,2023-08-16,2023-11-15,2024-02-15,2024-05-15,2024-08-15,39.590000,39.810001,39.669998,34.860001,38.730000,False,False,False
1232,Exelon Corporation,EXC,2023-09-15,248.48,Buy,EXC's continued growth environmental regulatio...,2023-09-16,2023-12-15,2024-03-15,2024-06-15,2024-09-15,41.650002,41.650002,35.490002,36.480000,35.540001,False,False,False
1233,Exelon Corporation,EXC,2023-10-15,148.36,Hold,EXC faces uncertainties shift consumer prefere...,2023-10-16,2024-01-15,2024-04-15,2024-07-15,2024-10-15,40.169998,40.169998,35.700001,36.270000,35.349998,False,False,False
1234,Exelon Corporation,EXC,2023-11-15,143.41,Buy,EXC benefiting labor shortages strong demand. ...,2023-11-16,2024-02-15,2024-05-15,2024-08-15,2024-11-15,39.669998,39.480000,34.860001,38.730000,37.500000,False,False,False


In [155]:
"""def calculate_performance(row):
    row["tp_reached_after_3_months"] = True if row["Target Price"] <= row["Price after 3 Months"] else False
    row["tp_reached_after_6_months"] = True if row["Target Price"] <= row["Price after 6 Months"] else False
    row["tp_reached_after_9_months"] = True if row["Target Price"] <= row["Price after 9 Months"] else False
    return row"""

In [156]:
testing_df = testing_df.apply(calculate_performance, axis=1)

TypeError: calculate_performance() got an unexpected keyword argument 'axis'

In [150]:
print()

0       None
1       None
2       None
3       None
4       None
        ... 
1231    None
1232    None
1233    None
1234    None
1235    None
Length: 1236, dtype: object

# Topic Modeling with LDA

# FINBERT

# KeyBERT

# LLM Argument Mining

- Provide List of pre-defined arguments